In [1]:
import glob
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import cv2
from PIL import Image
from numpy import asarray
import autokeras as ak
from tensorflow import keras
import os
from sklearn.model_selection import train_test_split

In [2]:
#create new dataframe
df = pd.DataFrame(columns=['image', 'label'])
df.head()

,image,label


In [3]:
df['image'] = os.listdir('data/')
df.head()

,image,label
0,CoreID101.tif,NaN
1,CoreID102.tif,NaN
2,CoreID105.tif,NaN
3,CoreID107.tif,NaN
4,CoreID108.tif,NaN


In [4]:
df_y = pd.read_csv('Only_Image_Y.csv')
df_y.head()

,SegmentDisplayName,CoreID,Tumor_Type,PtID,Sex,Age,Dx,Pri_Rec,IDH,Block,Row,Column
0,TMA 27 R1 | 007 | Geometric Segment,2,Invasive Edge,2,F,49.895,GBM,Primary,ND,NP-11-17045-B3,1,3
1,TMA 27 R1 | 015 | Geometric Segment,3,Pseudopalisading Necrosis,2,F,49.895,GBM,Recurrent,ND,NP-14-31147-A1,1,4
2,TMA 27 R1 | 028 | Geometric Segment,4,Pseudopalisading Necrosis,2,F,49.895,GBM,Recurrent,ND,NP-14-31147-A1,1,5
3,TMA 27 R1 | 038 | Geometric Segment,6,Cellular Tumor,3,F,66.683,GBM,Primary,ND,NP-13-26842-B1,1,7
4,TMA 27 R1 | 048 | Geometric Segment,7,Invasive Edge,3,F,66.683,GBM,Primary,ND,NP-13-26842-B1,1,8


In [5]:
df['label'] = df_y['Pri_Rec']
df.head()

,image,label
0,CoreID101.tif,Primary
1,CoreID102.tif,Recurrent
2,CoreID105.tif,Recurrent
3,CoreID107.tif,Primary
4,CoreID108.tif,Primary


In [6]:
df.to_csv('structured.csv')

In [7]:
#change all primary to 0 and recurrent to 1
df['label'] = df['label'].replace('Primary', 0)
df['label'] = df['label'].replace('Recurrent', 1)
df.head()

,image,label
0,CoreID101.tif,0
1,CoreID102.tif,1
2,CoreID105.tif,1
3,CoreID107.tif,0
4,CoreID108.tif,0


In [8]:
df_shuffled = df.sample(frac = 1)
df_shuffled.head()

,image,label
12,CoreID124.tif,0
40,CoreID24.tif,1
97,CoreID93.tif,1
80,CoreID70.tif,0
72,CoreID62.tif,1


In [9]:
df_shuffled.to_csv('shuffled_binned.csv', index=False)

In [10]:
train, test = train_test_split(df_shuffled, test_size=0.2)

In [11]:
train_x = train['image']
train_y = train['label']
test_x = test['image']
test_y = test['label']

In [12]:
def img_preprocess(img):
        """
        Opens the image and does some preprocessing
        such as converting to RGB, resize and converting to array
        """
        img = Image.open(img)
        img = img.convert('RGB')
        img = img.resize((256,256))
        img = asarray(img)/255

In [13]:
#get ndarray of images
train_x = np.array([np.array(Image.open('data/' + str(i))) for i in train_x])
test_x = np.array([np.array(Image.open('data/' + str(i))) for i in test_x])
train_x

array([[[[239, 240, 237],
         [238, 240, 239],
         [238, 240, 239],
         ...,
         [238, 239, 238],
         [238, 238, 237],
         [238, 238, 235]],

        [[238, 240, 239],
         [237, 240, 239],
         [238, 240, 239],
         ...,
         [237, 238, 239],
         [239, 238, 238],
         [238, 238, 237]],

        [[238, 240, 238],
         [239, 240, 238],
         [239, 241, 237],
         ...,
         [238, 238, 238],
         [239, 239, 239],
         [239, 238, 238]],

        ...,

        [[237, 236, 236],
         [237, 235, 236],
         [237, 236, 238],
         ...,
         [240, 240, 240],
         [240, 241, 240],
         [239, 240, 241]],

        [[237, 235, 238],
         [237, 235, 238],
         [237, 236, 237],
         ...,
         [238, 240, 238],
         [239, 240, 240],
         [237, 240, 240]],

        [[236, 235, 237],
         [236, 235, 236],
         [237, 236, 239],
         ...,
         [238, 240, 239],
        

In [14]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)
print(train_y.to_list())

(80, 728, 1064, 3)
(80,)
(20, 728, 1064, 3)
(20,)
[1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0]


In [15]:
# clf = ak.ImageClassifier(overwrite=True,
#     max_trials = 2)
# clf.fit(x = train_x, y = train_y, epochs = 10, validation_split = 0.3)
# autokeras
clf = ak.ImageClassifier(overwrite=True, max_trials=1,
    loss = 'binary_crossentropy')
clf.fit(x = train_x, y = train_y, epochs = 10, validation_split = 0.2)

Trial 1 Complete [00h 00m 47s]
val_loss: 0.5244350433349609

Best val_loss So Far: 0.5244350433349609
Total elapsed time: 00h 00m 47s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
Not enough memory, reduce batch size to 16.
Epoch 1/10
Not enough memory, reduce batch size to 8.
Epoch 1/10
10/10 [==============================] - 2s 193ms/step - loss: 100.4413 - accuracy: 0.5250
Epoch 2/10
10/10 [==============================] - 2s 187ms/step - loss: 3.6121 - accuracy: 0.6000
Epoch 3/10
10/10 [==============================] - 2s 187ms/step - loss: 0.7040 - accuracy: 0.7875
Epoch 4/10
10/10 [==============================] - 2s 188ms/step - loss: 0.1024 - accuracy: 1.0000
Epoch 5/10
10/10 [==============================] - 2s 187ms/step - loss: 0.0729 - accuracy: 1.0000
Epoch 6/10
10/10 [==============================] - 2s 186ms/step - loss: 0.0195 - accuracy: 1.0000
Epoch 7/10
10/10 [==============================] - 2s 188ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 8/10
10/10 

INFO:tensorflow:Assets written to: .\image_classifier\best_model\assets


INFO:tensorflow:Assets written to: .\image_classifier\best_model\assets


In [16]:
pred_test = clf.predict(test_x)
pred_test

1/1 [==============================] - 0s 31ms/step


array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.]], dtype=float32)

In [17]:
clf.evaluate(test_x, test_y)

1/1 [==============================] - 0s 203ms/step - loss: 0.8815 - accuracy: 0.4500


[0.8815237283706665, 0.44999998807907104]

In [18]:
print(test_y.tolist(), pred_test.tolist())

[1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1] [[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [0.0], [1.0], [1.0], [1.0], [0.0], [1.0], [1.0], [1.0], [1.0], [0.0]]
